In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lra-listops/basic_test.tsv
/kaggle/input/lra-listops/basic_train.tsv
/kaggle/input/listops/test_d20s.tsv
/kaggle/input/listops/train_d20s.tsv
/kaggle/input/validation/basic_val.tsv


### Imports

In [2]:
# Install the required version of transformers
!pip install -U transformers==4.40.2

# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from transformers import ReformerConfig, ReformerForSequenceClassification, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 72.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


### Data preview

In [ ]:
train_file = '/kaggle/input/listops/train_d20s.tsv'
test_file = '/kaggle/input/listops/test_d20s.tsv'

train_df = pd.read_csv(train_file, sep='\t', header=0)
test_df = pd.read_csv(test_file, sep='\t', header=0)
print(train_df.head())

train_df = pd.read_csv(train_file, sep='\t', header=0)
test_df = pd.read_csv(test_file, sep='\t', header=0)
print(train_df.head())
print(train_df.columns)


   Target                                             Source
0       6  ( ( ( ( ( [MAX ( ( ( ( ( ( [MED 4 ) 6 ) 6 ) 0 ...
1       7   ( ( ( ( [SM ( ( ( [MED 6 ) 5 ) ] ) ) 1 ) 1 ) ] )
2       4                 ( ( ( ( ( [MAX 3 ) 4 ) 3 ) 3 ) ] )
3       0  ( ( ( ( ( [MIN 0 ) 0 ) ( ( ( [MAX 4 ) ( ( ( ( ...
4       9  ( ( ( ( ( ( [SM ( ( ( ( [MIN 5 ) ( ( ( [MAX ( ...
   Target                                             Source
0       6  ( ( ( ( ( [MAX ( ( ( ( ( ( [MED 4 ) 6 ) 6 ) 0 ...
1       7   ( ( ( ( [SM ( ( ( [MED 6 ) 5 ) ] ) ) 1 ) 1 ) ] )
2       4                 ( ( ( ( ( [MAX 3 ) 4 ) 3 ) 3 ) ] )
3       0  ( ( ( ( ( [MIN 0 ) 0 ) ( ( ( [MAX 4 ) ( ( ( ( ...
4       9  ( ( ( ( ( ( [SM ( ( ( ( [MIN 5 ) ( ( ( [MAX ( ...
Index(['Target', 'Source'], dtype='object')


### Cleaning

In [ ]:
def clean_text(text):
    return ''.join(str(text).split())




In [6]:

# Remove any possible header rows included as data
train_df = train_df[train_df['Target'] != 'Target']
test_df = test_df[test_df['Target'] != 'Target']

# Convert labels to integers
train_df['Target'] = train_df['Target'].astype(int)
test_df['Target'] = test_df['Target'].astype(int)
print(train_df.head())
print("---------------")
for i in range(8) : 
    print(len(train_df['Source'][i]))

   Target                                             Source
0       6  ( ( ( ( ( [MAX ( ( ( ( ( ( [MED 4 ) 6 ) 6 ) 0 ...
1       7   ( ( ( ( [SM ( ( ( [MED 6 ) 5 ) ] ) ) 1 ) 1 ) ] )
2       4                 ( ( ( ( ( [MAX 3 ) 4 ) 3 ) 3 ) ] )
3       0  ( ( ( ( ( [MIN 0 ) 0 ) ( ( ( [MAX 4 ) ( ( ( ( ...
4       9  ( ( ( ( ( ( [SM ( ( ( ( [MIN 5 ) ( ( ( [MAX ( ...
---------------
73
48
34
271
501
577
72
172


In [ ]:
# Compute the lengths of the original sequences
seqLengths= train_df['Source'].apply(lambda x: len(x))

print("Training data sequence lengths:")
print(seqLengths.describe())

print(seqLengths.head())





Training data sequence lengths:
count    90000.000000
mean       277.778200
std        491.629979
min          1.000000
25%         49.000000
50%         99.000000
75%        259.000000
max       7593.000000
Name: Source, dtype: float64
0     73
1     48
2     34
3    271
4    501
Name: Source, dtype: int64


In [ ]:
import pandas as pd

train_df = train_df[train_df['Source'].apply(lambda x: len(x)) < 512]
test_df = test_df[test_df['Source'].apply(lambda x: len(x)) < 512]



# Check the filtered dataframe
seqLengths = train_df['Source'].apply(lambda x: len(x))

# Describe the sequence lengths
print("Training data sequence lengths:")
print(seqLengths.describe())

print(seqLengths.head())


Training data sequence lengths:
count    77650.000000
mean       124.938416
std        110.377955
min          1.000000
25%         43.000000
50%         81.000000
75%        165.000000
max        511.000000
Name: Source, dtype: float64
0     73
1     48
2     34
3    271
4    501
Name: Source, dtype: int64


In [9]:
# Save the sequences to a text file for tokenizer training
with open("listops_sequences.txt", "w") as f:
    for sequence in train_df["Source"]:
        f.write(sequence + "\n")


### Tokenizer

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize a WordLevel tokenizer
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))

# Set the pre-tokenization strategy
tokenizer.pre_tokenizer = Whitespace()

# Prepare a trainer with special tokens
trainer = WordLevelTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])

# Train the tokenizer on your text file
tokenizer.train(["listops_sequences.txt"], trainer)

# Save the tokenizer
tokenizer.save("custom_tokenizer.json")


In [11]:
from transformers import PreTrainedTokenizerFast

# Load the custom tokenizer
tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='', vocab_size=23, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


### Dataset

In [14]:
class LRADataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels.astype(int)  # Ensure labels are integers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        # Tokenize and encode the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = LRADataset(
        texts=df['Source'].to_numpy(),
        labels=df['Target'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=0
    )

MAX_LEN = 512
BATCH_SIZE = 8

# Create data loaders
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)


In [26]:
num_labels = train_df["Source"].nunique()
print(num_labels)
print(tokenizer.vocab_size)


77650
23


### Model Bert

In [ ]:
from transformers import MobileBertConfig, MobileBertForSequenceClassification

#  MobileBERT for the ListOps task
config = MobileBertConfig(
    vocab_size=23,
    hidden_size=32,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=128,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=512,
    num_labels=10,
    pad_token_id=0
)

# Initialize model and use single GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MobileBertForSequenceClassification(config)
model = model.to(device)



In [33]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)  # Wrap the model for multiple GPUs

model = model.to(device)

# Optimizer and scheduler
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=1e-3)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps // 10,
    num_training_steps=total_steps
)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss().to(device)


Using 2 GPUs


### Training

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        labels = batch["labels"].to(device, non_blocking=True)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss  #
        logits = outputs.logits
        _, preds = torch.max(logits, dim=1)

        # Update metrics
        correct_predictions += torch.sum(preds == labels)
        # Take mean if loss is not already reduced
        if loss.dim() > 0:
            loss = loss.mean()
        losses.append(loss.item())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device, non_blocking=True)
            attention_mask = batch["attention_mask"].to(device, non_blocking=True)
            labels = batch["labels"].to(device, non_blocking=True)

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            # Update metrics
            correct_predictions += torch.sum(preds == labels)
            if loss.dim() > 0:
                loss = loss.mean()
            losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

In [37]:
history = {
    'train_acc': [],
    'train_loss': [],
    'val_acc': [],
    'val_loss': []
}

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        test_data_loader,
        loss_fn,
        device
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')

    history['train_acc'].append(train_acc.cpu().numpy())
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.cpu().numpy())
    history['val_loss'].append(val_loss)


Epoch 1/5
----------


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Train loss 2.2690013328639487 accuracy 0.12798454603992274
Val   loss 2.309552396889086 accuracy 0.09759203519333179
Epoch 2/5
----------
Train loss 2.3084985762604284 accuracy 0.10588538312942691
Val   loss 2.305334810195146 accuracy 0.09770780273211391
Epoch 3/5
----------
Train loss 2.3055060945825057 accuracy 0.11103670315518352
Val   loss 2.305973263360836 accuracy 0.0985181755035888
Epoch 4/5
----------
Train loss 2.303169927403501 accuracy 0.11280103026400515
Val   loss 2.3029526266786786 accuracy 0.11102106969205834
Epoch 5/5
----------
Train loss 2.301761214613018 accuracy 0.1148486799742434
Val   loss 2.3004655679066977 accuracy 0.11102106969205834
